# Algorithmic Data Science: Lab 3 - Matrices: Partial Solution

## Exercise 1

A matrix can be represented as a list of lists, where each internal list is a row of the matrix

In [1]:
matrixA = [[1,3,4],[2,-1,0],[3,7,9]]
matrixB = [[0,6,1],[-1,-1,5],[3,0,-1]]
matrixC = [[2,3],[0,9]]
matrixD = [[1,4,5],[2,3],[1,7,9]]

Write functions to do the following. (Alternatively, you could write a matrix class which is initialised using a list of lists and then create methods for the following).

1. determine whether the input is a valid matrix
2. return the dimensionality of a valid matrix as (m,n) where m is the number of rows and n is the number of columns
3. transpose a matrix
4. add two compatible matrices
5. multiply a matrix by a scalar
6. multiply two compatible matrices using the naive method from the lectures
7. multiply two compatible matrices using Strassen's method (provided the matrices are nxn and n is a power of 2)

<b>Solution, using an object-oriented approach. Note you don't have to do it this way- you could more simply just use a bunch of functions and not use a class, but the following method is probably more educational for those wanting to learn more python. . .</b>

In [3]:
import numbers

def zeros(m,n):
    return [[0 for i in range(n)] for j in range(m)]


def dim(A):
    return (len(A), len(A[0]))


def validate_matrix(A):
    def row_len(A, i): 
        if type(A[i]) is not list:
            raise ValueError("not a list")
        for Aij in A[i]:
            if not isinstance(Aij, numbers.Number):
                raise ValueError("not a number")
        return len(A[i])
    try :
        if type(A) is not list:
            return False

        m = len(A)

        if m == 0:
            return False
        
        ni = row_len(A, 0)

        if ni == 0:
            return False
        elif ni == 1 and type(A[0]):
            return True

        nj = row_len(A, 1)
        i = 0
        while ni == nj and i < m - 2:
            i += 1
            ni = nj
            ni = row_len(A, i+1)
    except ValueError:
        return False
    return i == m-2


def transpose(M):
    
    m = len(M)
    n = len(M[0])
    
    C = [[0]*m for i in range(n)]
    for i in range(m):
        for j in range(n):
            C[j][i]=M[i][j]

    return C


def add(A, B):
    (m,n) = dim(A) 
    C = zeros(m,n)
    i = 0
    for (Ai, Bi) in zip(A, B):
        j = 0
        for (Aij, Bij) in zip(Ai, Bi):
            z = Aij + Bij
            C[i][j] = z
            j += 1 
        i += 1
            
    return C


def sub(A, B):
    (m,n) = dim(A) 
    C = zeros(m,n)
    i = 0
    for (Ai, Bi) in zip(A, B):
        j = 0
        for (Aij, Bij) in zip(Ai, Bi):
            z = Aij - Bij
            C[i][j] = z
            j += 1 
        i += 1
            
    return C


def mult(A, B):

    (m,n) = dim(A)
        
    C = zeros(m, m)
    
    for i in range(m):
        for j in range(m):
            for k in range(m):
                C[i][j] += A[i][k] * B[k][j]
        
    return C


def scalar_mult(A, b):
    (m,n) = dim(A)
    C = zeros(m,n) 
    i = 0
    for Ai in A:
        j = 0 
        for Aij in Ai:
            z = Aij * b
            C[i][j] = z
            j += 1
        i += 1
        
    return C


def segment(M):
    
    n = int(len(M[0]) / 2)
    
    a = [M[i][:n] for i in range(n)]
    b = [M[i][n:] for i in range(n)]
    c = [M[i][:n] for i in range(n, 2*n)]
    d = [M[i][n:] for i in range(n, 2*n)]
    
    return (a,b,c,d)


def build(a,b,c,d):
    
    C = [x[0] + x[1] for x in zip(a,b)] + [ y[0] + y[1] for y in zip(c,d)]
            
    return C
    

def strassen(M1, M2):
    
    if len(M1[0]) <= 2:
        return mult(M1,M2)
    
    (a,b,c,d) = segment(M1)

    #note: e g f h NOT e f g h 
    (e,g,f,h) = segment(M2)

    P1 = strassen(a, sub(g, h))
    P2 = strassen(add(a, b), h)
    P3 = strassen(add(c, d),e)
    P4 = strassen(d, sub(f, e))
    P5 = strassen(add(a, d), add(e, h))
    P6 = strassen(sub(b, d), add(f, h))
    P7 = strassen(sub(a, c), add(e, g))
    
    r = sub(add(add(P5, P4), P6), P2)
    s = add(P1, P2)
    t = add(P3, P4)
    u = add(sub(sub(P5, P3), P7), P1)
    
    C = build(r,s,t,u)
    
    return C


class Matrix:
    
    
    def __init__(self, m):
        self._m = m
        self.__validate()
    
    
    def __str__(self):
        return self._m.__str__()
    
    
#     def __repr__(self):
#         return self._m.__str__()
    
    
    def __validate(self):
        if not validate_matrix(self._m):
            raise ValueError("m is not a valid matrix")
            
    
    def dim(self):
        return dim(self._m)
    
    
    def square(self):
        (m,n) = self.dim()
        return m == n
    
    
    def ispower2(self):
        (m,n) = self.dim()
        if not m == n:
            return False
        power_of_2 = 1
        while power_of_2 <= n:
            if power_of_2 == n:
                return True
            power_of_2 *= 2
        return False
    
    
    def compatible(self, other):
        if other.dim() == self.dim():
            return True
        else :
            return False
    
    
    def __compatible(self, other):
        if not self.compatible(other):
            raise ValueError("arg has different dimensions")
        
    
    def add(self, other):
        self.__compatible(other)
        
        C = add(self._m, other._m)
        
        return Matrix(C)
    
    
    def sub(self, other):
        self.__compatible(other)
        
        C = sub(self._m, other._m)
        
        return Matrix(C)
    
    
    def transpose(self):
        T = transpose(self._m)
        return Matrix(T)
    
    
    def scalar_mult(self, n):        
        C = scalar_mult(self._m, n)
        
        return Matrix(C)
        
        
    def mult(self, other):
        self.__compatible(other)
        C = mult(self._m, other._m)

        return Matrix(C)
    
    
    def strassen(self, other):
        if not self.ispower2():
            raise ValueError("Dimension not a power of 2")
        C = strassen(self._m, other._m)
        return C
    
A = Matrix(matrixA)
B = Matrix(matrixB)
C = Matrix(matrixC)

###fail
#D = Matrix(matrixD)
# Matrix(["a"])
# Matrix([["a"]])
# Matrix([])

print(A.dim())

print(A.transpose())

print(A.sub(A).sub(A).add(A)) # zeros

print(B.scalar_mult(5))

E = Matrix([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]])

I = Matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])

print(A.mult(B))

print(E.mult(E))

print(E.strassen(E))

print(E.strassen(I.transpose())) # == E



(3, 3)
[[1, 2, 3], [3, -1, 7], [4, 0, 9]]
[[0, 0, 0], [0, 0, 0], [0, 0, 0]]
[[0, 30, 5], [-5, -5, 25], [15, 0, -5]]
[[9, 3, 12], [1, 13, -3], [20, 11, 29]]
[[90, 100, 110, 120], [202, 228, 254, 280], [314, 356, 398, 440], [426, 484, 542, 600]]
[[90, 100, 110, 120], [202, 228, 254, 280], [314, 356, 398, 440], [426, 484, 542, 600]]
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]


To get Strassen to beat the naive method you need to replace the following lines in the Strassen function:

In [ ]:
if len(M1[0]) <= 2:
        return mult(M1,M2)

with the following:

In [ ]:
if len(M1[0]) <= 45:
        return mult(M1,M2)